In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord, AltAz, Angle, EarthLocation

import astropy.units as u
from astropy.table import Table, vstack, join
from lsst.summit.utils import ConsDbClient
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import json

from lsst.geom import LinearTransform
from lsst.afw.geom.ellipses import Quadrupole
import matplotlib.pyplot as plt

In [ ]:
RUBIN_OBSERVATORY = EarthLocation(
    lat=Angle("-30d14m27.00s"),
    lon=Angle("-70d44m11.99s"),
    height=2722 * u.m,
)

In [ ]:
def parallactic_angle(coord, time):
    """Calculate the parallactic angle.

    Parameters
    ----------
    coord : astropy.coordinates.SkyCoord
        Coordinates of the object.  Must have alt and az attributes.
    time : astropy.time.Time
        Time of the observation.

    Returns
    -------
    astropy.coordinates.Angle
        Parallactic angle.
    """
    loc = RUBIN_OBSERVATORY
    coord = coord.transform_to(AltAz(obstime=time, location=loc))

    ha, dec = coord.hadec.ha, coord.hadec.dec
    lat = loc.lat

    sinq = np.sin(ha)
    cosq = np.tan(lat) * np.cos(dec) - np.sin(dec) * np.cos(ha)
    return Angle(np.arctan2(sinq, cosq))

In [ ]:
tokenfile = Path("~/.lsst/rsp_token").expanduser()
with open(tokenfile, "r") as f:
    token = f.read().strip()
cdb = ConsDbClient("https://usdf-rsp.slac.stanford.edu/consdb/", token=token)

In [ ]:
def get_stars(day_obs_min, day_obs_max):
    query = f"""
    SELECT
        visit1.day_obs,
        visit1.seq_num,
        detector,
        psf_ixx as ixx, psf_ixy as ixy, psf_iyy as iyy
    FROM
        cdb_lsstcam.visit1
    JOIN
        cdb_lsstcam.ccdvisit1 ON cdb_lsstcam.visit1.visit_id = cdb_lsstcam.ccdvisit1.visit_id
    JOIN
        cdb_lsstcam.ccdvisit1_quicklook ON cdb_lsstcam.ccdvisit1.ccdvisit_id = cdb_lsstcam.ccdvisit1_quicklook.ccdvisit_id
    WHERE
        cdb_lsstcam.visit1.day_obs >= {day_obs_min} and cdb_lsstcam.visit1.day_obs < {day_obs_max}
    """
    stars = cdb.query(query)
    stars["day_obs"] = stars["day_obs"].astype(int)
    stars["seq_num"] = stars["seq_num"].astype(int)
    stars["detector"] = stars["detector"].astype(int)
    stars["ixx"] = stars["ixx"].astype(float)
    stars["ixy"] = stars["ixy"].astype(float)
    stars["iyy"] = stars["iyy"].astype(float)
    
    good = (
        np.isfinite(stars["ixx"]) 
        & np.isfinite(stars["ixy"]) 
        & np.isfinite(stars["iyy"])
    )
    stars = stars[good]
    
    return stars

In [ ]:
# months = [8]
months = [8, 9, 10, 11, 12]
days = np.round(np.linspace(0, 32, 32)).astype(int)
stars = []
for month in tqdm(months, desc="Months"):
    for i in tqdm(range(len(days)-1), desc="Days", leave=False):
        start = int(f"2025{month:02d}{days[i]:02d}")
        stop = int(f"2025{month:02d}{days[i+1]:02d}")
        try:
            s = get_stars(start, stop)
        except ValueError:
            continue
        stars.append(s)
stars = vstack(stars)
# Add space for az/alt rotated moments
stars["rot_ixx"] = np.nan
stars["rot_ixy"] = np.nan
stars["rot_iyy"] = np.nan

stars = stars.group_by(["day_obs", "seq_num"])

In [ ]:
query = """
SELECT
    day_obs,
    seq_num,
    azimuth,
    altitude,
    s_ra,
    s_dec,
    sky_rotation,
    zenith_distance,
    exp_midpt_mjd
FROM
    cdb_lsstcam.visit1
WHERE
    day_obs >= 20250801
"""
exposures = cdb.query(query)

In [ ]:
for i, key in enumerate(tqdm(stars.groups.keys)):
    start, end = stars.groups.indices[i], stars.groups.indices[i+1]
    row = stars[start]
    day_obs = row["day_obs"]
    seq_num = row["seq_num"]

    exposure_select = (
        (exposures["day_obs"] == day_obs)
        & (exposures["seq_num"] == seq_num)
    )
    exposure_row = exposures[exposure_select][0]

    mjd = exposure_row["exp_midpt_mjd"]
    t = Time(mjd, format="mjd", scale="tai")
    az = exposure_row["azimuth"] * u.deg
    alt = exposure_row["altitude"] * u.deg
    target = SkyCoord(az=az, alt=alt, frame=AltAz(obstime=t, location=RUBIN_OBSERVATORY))
    q = parallactic_angle(target, t)

    rsp = exposure_row["sky_rotation"] * u.deg
    rtp = q - rsp - 90 * u.deg
    srtp, crtp = np.sin(rtp), np.cos(rtp)
    rot = np.array([[crtp, srtp], [-srtp, crtp]]) @ np.array([[0, 1], [1, 0]]) @ np.array([[-1, 0], [0, 1]])

    transform = LinearTransform(rot)
    rotShapes = []
    for row in stars[start:end]:
        shape = Quadrupole(row["ixx"], row["iyy"], row["ixy"])
        rotShape = shape.transform(transform)
        rotShapes.append(rotShape)
    stars["rot_ixx"][start:end] = [sh.getIxx() for sh in rotShapes]
    stars["rot_ixy"][start:end] = [sh.getIxy() for sh in rotShapes]
    stars["rot_iyy"][start:end] = [sh.getIyy() for sh in rotShapes]

In [ ]:
tables = []
files = list(Path("guiders").glob("*2025*.json"))
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
    dayobs = int(file.name[7:15])
    
    seqnums = sorted(map(int, data.keys()))
    # Have to loop through all rows to get all possible columns
    cols = set()
    for row in data.values():
        cols.update([k for k in row if not k.startswith("_")])
    cols = sorted(cols)
    vals = {col: [data[str(seq)].get(col, float("nan")) for seq in seqnums] for col in cols}
    vals["seqnum"] = seqnums
    table = Table(vals)
    for col in cols:
        if isinstance(table[col].dtype, object):
            for t in [float, str]:
                try:
                    table[col] = table[col].astype(t)
                except:
                    continue
                else:
                    break
    table["dayobs"] = int(dayobs)
    tables.append(table)
guider_table = vstack(tables)

In [ ]:
table = join(stars, guider_table, keys_left=["day_obs", "seq_num"], keys_right=["dayobs", "seqnum"])
good = np.isfinite(table["Az drift (arcsec total)"]) & np.isfinite(table["Alt drift (arcsec total)"])
table = table[good]

In [ ]:
table["dx"] = table["Az drift (arcsec total)"] * table["Exposure time"] / 0.2  # arcsec -> pix
table["dy"] = table["Alt drift (arcsec total)"] * table["Exposure time"] / 0.2

In [ ]:
table["dxx"] = table["dx"] * table["dx"] / 12
table["dxy"] = table["dx"] * table["dy"] / 12
table["dyy"] = table["dy"] * table["dy"] / 12

In [ ]:
table["dphase"] = -0.5 * np.arctan2(2*table["dxy"], table["dxx"] - table["dyy"])

In [ ]:
d = np.array([[table["dxx"], table["dxy"]], [table["dxy"], table["dyy"]]])
d2 = np.array([[table["rot_ixx"], table["rot_ixy"]], [table["rot_ixy"], table["rot_iyy"]]])
cph = np.cos(table["dphase"])
sph = np.sin(table["dphase"])
r = np.array([[cph, -sph], [sph, cph]])

In [ ]:
out = np.einsum("iac,abc,jbc->ijc", r, d, r)
out2 = np.einsum("iac,abc,jbc->ijc", r, d2, r)
drift = out[0,0]
plus = out2[0,0] - out2[1,1]
cross = 2*out2[0,1]

In [ ]:
plt.figure(figsize=(5, 5))
plt.hexbin(drift, plus, extent=[0, 2, -2, 2])
plt.xlabel("drift moment [arcsec^2]")
plt.ylabel("+ PSF moment along drift axis [arcsec^2]")
# plt.plot([0, 2], [0, 2], c="r")
# plt.plot([0, 2], [0, 0], c="r")
plt.savefig("plus.png", dpi=200)
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))
plt.hexbin(drift, cross, extent=[0, 2, -2, 2])
plt.xlabel("drift moment [arcsec^2]")
plt.ylabel("x PSF moment along drift axis [arcsec^2]")
# plt.plot([0, 2], [0, 2], c="r")
# plt.plot([0, 2], [0, 0], c="r")
plt.savefig("cross.png", dpi=200)
plt.show()